In [2]:
import os
import numpy as np
import keras.backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler,Normalizer
import pandas as pd
import sklearn.svm as svm
from imblearn.datasets import make_imbalance
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix

df = pd.DataFrame(columns=['Parameter','fold1','fold2','fold3','fol4'])
#for i in range(1,80):
#for i in range(1,25):
Accuracy = np.array('Accuracy')
Prec = np.array('Precision')
Sens = np.array('Sensitivity')
Spec = np.array('Specificity')

for fold in range(1,5):
    features = pd.read_csv('ICHFeatures.csv',header=0)
    OS_train = pd.read_csv('Fold'+str(fold)+'/Train_dir.txt',header = None, dtype=str)
    OS_train.columns = ['ID','OS']
    OS_valid = pd.read_csv('Fold'+str(fold)+'/Val_Dir.txt',header = None, dtype=str)
    OS_valid.columns = ['ID','OS']
    features['ID']=features['ID'].str.replace('ct1','')
    train = pd.merge(features, OS_train, how='right', on='ID')
    test =  pd.merge(features, OS_valid, how='right', on='ID')
    norm_wihtout = [col for col in train.columns if col not in ['ID','GCS','Onset','OS']]
    #norm_valid = [col for col in test.columns if col not in ['ID','GCS','Onset','OS']]
    scaler = StandardScaler()
    train_ss = scaler.fit_transform(train[norm_wihtout])
    test_ss = scaler.transform(test[norm_wihtout])
    train[norm_wihtout] = train_ss
    test[norm_wihtout] = test_ss
    #train = train.assign(norm_train.values = train_ss)
    col_withoutID = [col for col in train.columns if col not in ['ID','OS']]
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(train[col_withoutID], train['OS'].values.astype(int))
    train_X = pd.DataFrame(X_res,columns = col_withoutID)
    train_y = pd.DataFrame(y_res, columns = ['OS'])
    num_features = 24
    estimator = svm.SVC(kernel="linear", C=1) #
    print(num_features)
    rfe=RFE(estimator,  n_features_to_select=num_features,step=1)
    rfe.fit(train_X,train_y)
    ranking_RFE=rfe.ranking_
    indices=np.where(ranking_RFE==1)
    indices = list(indices[0])
    data_RFE=train_X.iloc[:,indices]
    valid_RFE = test[col_withoutID].iloc[:,indices]
    print(data_RFE.columns)
    model = svm.SVC(kernel="linear", C=1) #LinearRegression()
    model.fit(data_RFE, train_y)

    Y_pred=model.predict(valid_RFE).ravel()
    acc=metrics.accuracy_score(test['OS'].values.astype(int),Y_pred)
    print("accuracy score = "+str(acc)) 
    Accuracy = np.append(Accuracy,acc)
    con_matrix = confusion_matrix(test['OS'].values.astype(int).tolist(),Y_pred.tolist())
    TN,FP,FN,TP = con_matrix.ravel()
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    #Precision
    PPV = TP/(TP+FP)
    Prec = np.append(Prec,PPV)
    Sens = np.append(Sens,TPR)
    Spec = np.append(Spec,TNR)
    
Accuracy = pd.DataFrame(data = Accuracy.reshape(1,5),columns = df.columns)
Prec = pd.DataFrame(data = Prec.reshape(1,5),columns = df.columns)
Sens = pd.DataFrame(data = Sens.reshape(1,5),columns = df.columns)
Spec = pd.DataFrame(data = Spec.reshape(1,5),columns = df.columns)
df = df.append(Accuracy)
df = df.append(Prec)
df = df.append(Sens)
df= df.append(Spec)
#del Accuracy

24
Index(['Age', 'GCS', 'Onset', 'CTA', 'Centroid1', 'Centroid2', 'Centroid3',
       'MajorAxisLength', 'FirstAxis1', 'FirstAxis2', 'FirstAxis3',
       'SecondAxis2', 'SecondAxis3', 'ThirdAxis2', 'ThirdAxis3', 'Eigen2',
       'kurtosis', 'histogram', 'bb1', 'bb3', 'entropy', 'f1', 'f2', 'f3'],
      dtype='object')


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy score = 0.7
24


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

Index(['Age', 'GCS', 'Onset', 'CTA', 'MinorAxisLength', 'FirstAxis1',
       'FirstAxis2', 'FirstAxis3', 'SecondAxis1', 'SecondAxis3', 'ThirdAxis1',
       'ThirdAxis2', 'ThirdAxis3', 'Eigen2', 'Eigen3', 'kurtosis', 'bb5',
       'bb6', 'extent', 'diameter', 'f1', 'f3', 'f4', 'f5'],
      dtype='object')
accuracy score = 0.6
24
Index(['Age', 'CTA', 'Centroid3', 'MajorAxisLength', 'FirstAxis1',
       'FirstAxis2', 'FirstAxis3', 'SecondAxis1', 'SecondAxis2', 'SecondAxis3',
       'ThirdAxis1', 'ThirdAxis2', 'Eigen3', 'kurtosis', 'histogram', 'bb1',
       'bb6', 'extent', 'diameter', 'solidity', 'f1', 'f2', 'f3', 'f5'],
      dtype='object')
accuracy score = 0.8
24
Index(['Age', 'GCS', 'CTA', 'Centroid1', 'FirstAxis1', 'FirstAxis2',
       'FirstAxis3', 'SecondAxis1', 'SecondAxis2', 'SecondAxis3', 'ThirdAxis1',
       'ThirdAxis2', 'ThirdAxis3', 'Eigen2', 'Eigen3', 'kurtosis', 'histogram',
       'bb1', 'bb2', 'bb4', 'f1', 'f3', 'f4', 'f5'],
      dtype='object')
accuracy score = 0.75


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [3]:
df

,Parameter,fold1,fold2,fold3,fol4
0,Accuracy,0.7,0.6,0.8,0.75
0,Precision,0.0,0.2857142857142857,0.6666666666666666,0.5
0,Sensitivity,0.0,0.4,0.4,0.4
0,Specificity,0.9333333333333333,0.6666666666666666,0.9333333333333333,0.8666666666666667


In [ ]:
df =pd.concat([df,df.iloc[:,1:].mean(axis=1)],axis = 1)
df.columns = ['num_features','fold1','fold2','fold3','fol4','Average']

In [4]:
df.to_csv('SVM_results.csv')

In [ ]:
from scipy.stats import pearsonr
corr, _ = pearsonr(train['kurtosis'].astype(int), train['OS'].astype(int))
print('Pearsons correlation: %.3f' % corr)

In [ ]:
from scipy.stats import pearsonr
corr, _ = pearsonr(train['SecondAxis2'].astype(int), train['OS'].astype(int))
print('Pearsons correlation: %.3f' % corr)

In [ ]:
corr, _ = pearsonr(train['ThirdAxis1'].astype(int), train['OS'].astype(int))
print('Pearsons correlation: %.3f' % corr)

In [ ]:
corr, _ = pearsonr(train['Age'].astype(int), train['OS'].astype(int))
print('Pearsons correlation: %.3f' % corr)

In [ ]:
import shap
import sklearn.svm as svm
 


In [ ]:
print(Y_pred)
print(test['OS'].values)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(test['OS'].values,Y_pred)
# FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
# FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
# TP = np.diag(confusion_matrix)
# TN = confusion_matrix.sum() - (FP + FN + TP)

TN,FP,FN,TP = confusion_matrix.ravel()
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
#Precision
PPV = TP/(TP+FP)

print('Sensitivity ='+ str(TPR))
print('Specificity = ' +str(TNR))
print('Precision = '+str(PPV))

In [ ]:
print(confusion_matrix)

In [ ]:
print(FP)

In [ ]:
import shap
explainer = shap.LinearExplainer(model, data_RFE, feature_dependence="independent")
shap_values = explainer.shap_values(data_RFE)
shap.initjs()
# plot the SHAP values for the Setosa output of the first instance
shap.summary_plot(shap_values, data_RFE.iloc[:,:], feature_names=data_RFE.columns,show=False)
plt.savefig('SVM_fold4.png')